# Quantification of shape during growth of gemmae

## Documentation and Imports


Created on 01-04-2021

Author: Valentin Laplaud

This code is performing measurements on the gemma contours computed by 'ShapeParametrisationNB'

This light version has only a few quantification used regularly, the complete version is an archive of all tested quantifications.

In [ ]:
## Clean up before script start 

for element in dir():

    if element[0:1] != "_":

        del globals()[element]

import gc
gc.collect()


print('\033[1m' + '\033[4m' + '\nRunning ''ShapeQuantificationNB_Light'' \n' + '\033[0m')

# plotting stuff
import matplotlib as mpl
mpl.use('TkAgg')
%matplotlib inline

COLOR = 'white'
COLOR2 = 'black'

mpl.rcParams['text.color'] = COLOR
mpl.rcParams['axes.labelcolor'] = COLOR
mpl.rcParams['xtick.color'] = COLOR
mpl.rcParams['ytick.color'] = COLOR
mpl.rcParams['axes.edgecolor'] = COLOR

mpl.rcParams["figure.facecolor"] = COLOR2
mpl.rcParams["axes.facecolor"] = COLOR2
mpl.rcParams["savefig.facecolor"] = COLOR2
mpl.rcParams['axes.facecolor'] = COLOR2

import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
import seaborn as sns
from cycler import cycler
#Default colors
colorcycle = [plt.get_cmap('viridis')(1. * i/10) for i in range(10)]
mpl.rcParams['axes.prop_cycle'] = cycler(color=colorcycle)

# numbers handling
import numpy as np
import pandas as pd

# signal processing 
from scipy.signal import savgol_filter, correlate, correlation_lags
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit, least_squares
from scipy.stats import ranksums

# images handling
from skimage import io
from skimage.filters import threshold_otsu, gaussian, laplace, sobel
from skimage.measure import label, regionprops, regionprops_table
from skimage.util import invert
from skimage.morphology import binary_opening, binary_closing, remove_small_holes,binary_erosion
from skimage.color import rgb2gray
from skimage.segmentation import active_contour, morphological_geodesic_active_contour,morphological_chan_vese, checkerboard_level_set, inverse_gaussian_gradient
import cv2 as cv

# to hide known warnings
import warnings

# General system functions
import os
import shutil
import sys

import time

# my functions
sys.path.append('../')
import VallapFunc as vf


In [ ]:
# my functions
sys.path.append('../')
import VallapFunc as vf

## Define analysis functions

###  Mean contour
With and without alignement in time based on dormancy exit 

Contours computed when at least 20 gemmae are averageable

In [ ]:
def computeMeanContourTime(CD,GD,Tstarts):
    StackList = np.unique(Tstarts.index)
    
    meanCD = pd.DataFrame(data=None,columns = ['X','Y','Img'])
    meanGD = pd.DataFrame(data=None,columns = ['nppgs','Img'])
    
    # number of contour points
    npts = len(CD.loc[(CD.index == StackList[0]) & (CD['Img'] == 0),'X'])
    
############################## Contours averaged at each time point ###################################################    
    cnt = 0 # number of mean contours done
    
    SL = StackList[:]
    
    nppgs = len(SL)
    
    while nppgs>19:
        
        print('Computing mean contour n°' + str(cnt+1) + ' :')
        
        # Storing variables
        Xs = np.empty((npts,nppgs))
        Xs[:] = np.nan
        Ys = np.empty((npts,nppgs))
        Ys[:] = np.nan


        for s,i in zip(SL,range(len(SL))) :

            Xs[:,i] = CD.loc[(CD.index==s) & (CD['Img'] == int(cnt)), 'X']
            Ys[:,i] = CD.loc[(CD.index==s) & (CD['Img'] == int(cnt)), 'Y']
            

        meanX = np.mean(Xs,axis=1)
        meanY = np.mean(Ys,axis=1)           

        print(str(nppgs) + ' contours averaged together.\n')

        data = {'X':meanX,
                    'Y':meanY,
                    'Img':cnt*np.ones(len(meanX))} 

        newdata = pd.DataFrame(data=data,index = np.repeat('meanCnotal',len(meanX)))

        meanCD = meanCD.append(newdata) # adding to global dataframe
        
        data = {'nppgs':nppgs,
            'Img':cnt} 

        newdata = pd.DataFrame(data=data,index = ['meanCnotal'])
        
        meanGD = meanGD.append(newdata)
        
        
        cnt = cnt+1
        newSL = []
        for s in SL :
            if np.max(CD.loc[s,'Img'])-cnt>0:
                newSL.append(s)
        
        SL = newSL[:]
        
        nppgs = len(SL)
        

 ################################ Contours aligned in time based on dormancy exit ####################################
    cnt = 0 # number of mean contours done
    
    SL = StackList[:]
    
    nppgs = len(SL)

    while nppgs>19:

        print('Computing mean contour (aligned in time) n°' + str(cnt+1) + ' :')

        # Storing variables
        Xs = np.empty((npts,nppgs))
        Xs[:] = np.nan
        Ys = np.empty((npts,nppgs))
        Ys[:] = np.nan


        for s,i in zip(SL,range(len(SL))) :

            Xs[:,i] = CD.loc[(CD.index==s) & (CD['Img'] == int(cnt+Tstarts[s])), 'X']
            Ys[:,i] = CD.loc[(CD.index==s) & (CD['Img'] == int(cnt+Tstarts[s])), 'Y']
            
            GD.loc[(GD.index==s) & (GD['Img'] == int(cnt+Tstarts[s])),'MeanCimg'] = cnt

        meanX = np.mean(Xs,axis=1)
        meanY = np.mean(Ys,axis=1)           

        print(str(nppgs) + ' contours averaged together.\n')

        data = {'X':meanX,
                    'Y':meanY,
                    'Img':cnt*np.ones(len(meanX))} 

        newdata = pd.DataFrame(data=data,index = np.repeat('meanC',len(meanX)))

        meanCD = meanCD.append(newdata) # adding to global dataframe
        
        data = {'nppgs':nppgs,
            'Img':cnt} 

        newdata = pd.DataFrame(data=data,index = ['meanC'])
        
        meanGD = meanGD.append(newdata)
        
        
        cnt = cnt+1
        newSL = []
        for s in SL :
            if np.max(CD.loc[s,'Img'])-Tstarts[s]-cnt>0:
                newSL.append(s)
        
        SL = newSL[:]
        
        nppgs = len(SL)
    
    return(meanCD,meanGD)
            

print('Function defined.')

### Distance to mean contours

In [ ]:
def DistToMean(Pfig,CD,GD,CDmean,Expname,**kwargs):
    
    ## Params
    StackList = np.unique(GD.index)
    
    ## Kwargs    
    showPlots = False
    doPlots = False
    
    for key, value in kwargs.items(): 
        if key == 'doPlots':
            doPlots = value
            showPlots = value
        elif key == 'showPlots':
            showPlots = value         
        else:
            print('Unknown key : ' + key + '. Kwarg ignored.')
    
    ## Saving folders
    if not os.path.exists(Pfig + '\\MeanContoursVariability\\'):
        os.mkdir(Pfig + '\\MeanContoursVariability\\') # create general folder  
    
    fullP = Pfig + '\\MeanContoursVariability\\' + Expname + '\\'
    
    if not os.path.exists(fullP):
            os.mkdir(fullP) # create experiment folder 
    
    if not os.path.exists(fullP + '\\OriginalContours\\'):
            os.mkdir(fullP + '\\OriginalContours\\') # create experiment folder  
    
    if not os.path.exists(fullP + '\\NormalizedContours\\'):
            os.mkdir(fullP + '\\NormalizedContours\\') # create experiment folder    
              
############################## Contours averaged at each time point ###################################################      
    nimg = int(np.max(CDmean.loc['meanCnotal','Img']))
    
    for i in range(nimg+1):
        
        print('Computing variability around mean contours (not time aligned) n°' + str(i))
        
        Xmean = CDmean.loc[(CDmean.index == 'meanCnotal') & (CDmean['Img']==i),'X'].values
        Ymean = CDmean.loc[(CDmean.index == 'meanCnotal') & (CDmean['Img']==i),'Y'].values
        
        # normalized mean contour
        XmeanN,YmeanN = vf.normaliseContourByArea(Xmean,Ymean)
        
        if doPlots:                        
            fig1,ax1 = plt.subplots(dpi=300)
            fig1.suptitle(Expname + ' contours+mean (not time aligned) img n°' + str(i))
            ax1.set_aspect('equal', adjustable='box')
            ax1.set_xlabel('X (µm)')
            ax1.set_ylabel('Y (µm)')
            
            
            boundX = [np.max(CDmean.loc[(CDmean.index == 'meanCnotal') & (CDmean['Img']==i),'X'])*1.5,
                      np.min(CDmean.loc[(CDmean.index == 'meanCnotal') & (CDmean['Img']==i),'X'])*1.5]
            boundY = [np.max(CDmean.loc[(CDmean.index == 'meanCnotal') & (CDmean['Img']==i),'Y'])*1.5,
                      np.min(CDmean.loc[(CDmean.index == 'meanCnotal') & (CDmean['Img']==i),'Y'])*1.5]
            
            ax1.plot(boundX,boundY,'ko',ms = 1,zorder = -2)
            
            ax1.plot(Xmean,Ymean,'r-',lw= 2,zorder=0)
            
            fig2,ax2 = plt.subplots(dpi=300)
            fig2.suptitle(Expname + ' normalized contours+mean (not time aligned) img n°' + str(i))
            ax2.set_aspect('equal', adjustable='box')
            ax2.set_xlabel('X (norm)')
            ax2.set_ylabel('Y (norm)') 
            ax2.set_xlim(-0.8,0.8)
            ax2.set_ylim(-0.8,0.8)
        
            ax2.plot(XmeanN,YmeanN,'r-',lw= 2,zorder=0)
        
        for s in StackList:
            
            if not CD.loc[ (CD.index==s) & (CD['Img'] == i ),'X'].values.size==0:                
                
                X = CD.loc[ (CD.index==s) & (CD['Img'] == i ),'X'].values
                Y = CD.loc[ (CD.index==s) & (CD['Img'] == i ),'Y'].values
                
                AllDists = vf.dist(X,Y,Xmean,Ymean)
                MedDist = np.nanmedian(AllDists);
                L2Dist = np.sqrt(np.sum(np.square(AllDists)))/len(AllDists)
                
                GD.loc[(GD.index==s) & (GD['Img']==i),'NotalMedDist'] = MedDist
                GD.loc[(GD.index==s) & (GD['Img']==i),'NotalL2Dist'] = L2Dist
                
                # normalized mean contour
                XN,YN = vf.normaliseContourByArea(X,Y)
                
                AllDistsN = vf.dist(XN,YN,XmeanN,YmeanN)
                MedDistN = np.nanmedian(AllDistsN);
                L2DistN = np.sqrt(np.sum(np.square(AllDistsN)))/len(AllDistsN)
                
                GD.loc[(GD.index==s) & (GD['Img']==i),'NotalMedDistNorm'] = MedDistN
                GD.loc[(GD.index==s) & (GD['Img']==i),'NotalL2DistNorm'] = L2DistN
            
                if doPlots:
                    ax1.plot(X,Y,'w-',lw=0.3,zorder=-1)
                    ax2.plot(XN,YN,'w-',lw=0.3,zorder=-1)
        
        
        if doPlots:
            fig1.savefig(fullP + '\\OriginalContours\\' + 'NotalMeanAndVar_Img' + str(i)  + '.png')
            fig2.savefig(fullP + '\\NormalizedContours\\' + 'Norm_NotalMeanAndVar_Img' + str(i)  + '.png')
            
        
            if showPlots:
                plt.show() 
            else:
                plt.close(fig1)
                plt.close(fig2)
                
                
 ################################ Contours aligned in time based on dormancy exit ####################################    
    nimg = int(np.max(CDmean.loc['meanC','Img']))
    
    for i in range(nimg+1):
        
        print('Computing variability around mean contours n°' + str(i))
        
        Xmean = CDmean.loc[(CDmean.index == 'meanC') & (CDmean['Img']==i),'X'].values
        Ymean = CDmean.loc[(CDmean.index == 'meanC') & (CDmean['Img']==i),'Y'].values
        
        # normalized mean contour
        XmeanN,YmeanN = vf.normaliseContourByArea(Xmean,Ymean)
        
        if doPlots:                        
            fig1,ax1 = plt.subplots(dpi=300)
            fig1.suptitle(Expname + ' contours+mean img n°' + str(i))
            ax1.set_aspect('equal', adjustable='box')
            ax1.set_xlabel('X (µm)')
            ax1.set_ylabel('Y (µm)')
            
            
            boundX = [np.max(CDmean.loc[(CDmean.index == 'meanC') & (CDmean['Img']==i),'X'])*1.5,
                      np.min(CDmean.loc[(CDmean.index == 'meanC') & (CDmean['Img']==i),'X'])*1.5]
            boundY = [np.max(CDmean.loc[(CDmean.index == 'meanC') & (CDmean['Img']==i),'Y'])*1.5,
                      np.min(CDmean.loc[(CDmean.index == 'meanC') & (CDmean['Img']==i),'Y'])*1.5]
            
            ax1.plot(boundX,boundY,'ko',ms = 1,zorder = -2)
            
            ax1.plot(Xmean,Ymean,'r-',lw= 2,zorder=0)
            
            fig2,ax2 = plt.subplots(dpi=300)
            fig2.suptitle(Expname + ' normalized contours+mean img n°' + str(i))
            ax2.set_aspect('equal', adjustable='box')
            ax2.set_xlabel('X (norm)')
            ax2.set_ylabel('Y (norm)') 
            ax2.set_xlim(-0.8,0.8)
            ax2.set_ylim(-0.8,0.8)
        
            ax2.plot(XmeanN,YmeanN,'r-',lw= 2,zorder=0)
        
        for s in StackList:
            
            if not GD.loc[(GD.index==s)&(GD['MeanCimg']==i),'Img'].values.size==0:
                
                AlgImg = GD.loc[ (GD.index==s) & (GD['MeanCimg']==i),'Img'].values[0]
                
                X = CD.loc[ (CD.index==s) & (CD['Img'] == AlgImg ),'X'].values
                Y = CD.loc[ (CD.index==s) & (CD['Img'] == AlgImg ),'Y'].values
                
                AllDists = vf.dist(X,Y,Xmean,Ymean)
                MedDist = np.nanmedian(AllDists);
                L2Dist = np.sqrt(np.sum(np.square(AllDists)))/len(AllDists)
                
                GD.loc[(GD.index==s) & (GD['MeanCimg']==i),'MedDist'] = MedDist
                GD.loc[(GD.index==s) & (GD['MeanCimg']==i),'L2Dist'] = L2Dist
                
                # normalized mean contour
                XN,YN = vf.normaliseContourByArea(X,Y)
                
                AllDistsN = vf.dist(XN,YN,XmeanN,YmeanN)
                MedDistN = np.nanmedian(AllDistsN);
                L2DistN = np.sqrt(np.sum(np.square(AllDistsN)))/len(AllDistsN)
                
                GD.loc[(GD.index==s) & (GD['MeanCimg']==i),'MedDistNorm'] = MedDistN
                GD.loc[(GD.index==s) & (GD['MeanCimg']==i),'L2DistNorm'] = L2DistN
            
                if doPlots:
                    ax1.plot(X,Y,'w-',lw=0.3,zorder=-1)
                    ax2.plot(XN,YN,'w-',lw=0.3,zorder=-1)
        
        
        if doPlots:
            fig1.savefig(fullP + '\\OriginalContours\\' + 'MeanAndVar_Img' + str(i)  + '.png')
            fig2.savefig(fullP + '\\NormalizedContours\\' + 'Norm_MeanAndVar_Img' + str(i)  + '.png')
            
        
            if showPlots:
                plt.show() 
            else:
                plt.close(fig1)
                plt.close(fig2)
            
    

## Plot fuctions

### Plot evolution of individual contours in time.

In [ ]:
def plotIndiContours(Pfig,CD, Expname, **kwargs):
    ## Params
    StackList = np.unique(CD.index)
    
    Plots = False
    
    for key, value in kwargs.items(): 
        if key == 'showplots':
            Plots = value
        else:
            print('Unknown key : ' + key + '. Kwarg ignored.')
    
    if not os.path.exists(Pfig + '\\IndividualContours\\'):
        os.mkdir(Pfig + '\\IndividualContours\\') # create general folder  
    
    fullP = Pfig + '\\IndividualContours\\' + Expname + '\\'
    
    if not os.path.exists(fullP):
            os.mkdir(fullP) # create experiment folder        
            
    for s in StackList:
        
        if not os.path.exists(fullP + '\\' + s + '\\'):
            os.mkdir(fullP + '\\' + s + '\\') # create gemmae folder 
            
        nimg = int(np.max(CD.loc[s, 'Img']))
        
        fig1, ax1 = plt.subplots(dpi=350)
        fig1.suptitle(s + ' contours in time.')
        ax1.set_aspect('equal', adjustable='box')
        ax1.set_xlabel('X (µm)')
        ax1.set_ylabel('Y (µm)')
        
        # plot invisible X,Y extremes to set the figure size
        X = [CD.loc[s,'X'].min(),CD.loc[s,'X'].max()]
        Y = [CD.loc[s,'Y'].min(),CD.loc[s,'Y'].max()]
        ax1.plot(X,Y,color = 'k')
        
        colorcycle = [plt.get_cmap('plasma')(1. * i/nimg) for i in range(nimg)]
        
        for i in range(nimg):

            X = CD.loc[(CD.index == s) & (CD['Img'] == i),'X']  
            Y = CD.loc[(CD.index == s) & (CD['Img'] == i),'Y']

            ax1.plot(X,Y,color = colorcycle[i])
            
            fig1.savefig(fullP  + '\\' + s + '\\' + s + ' - ' + str(i)  + '.png')
            
        
        if Plots:
            plt.show()
        else:
            plt.close(fig1)
        
        
        

### Plot distances to mean contour

In [ ]:
def plotMeanDist(Pfig,GDs,Expname,colors,Times):
    
    Pfolder = Pfig + '\\MeanContoursVariability\\'
    if not os.path.exists(Pfolder):
        os.mkdir(Pfolder) # create general folder  
    ## Evolution in time per experiment
    for j in range(len(Expname)):
        GD = GDs[j]
        
        ## Saving folders
        fullP = Pfolder + Expname[j] + '\\'

        if not os.path.exists(fullP):
                os.mkdir(fullP) # create experiment folder  

        ## Params
        StackList = np.unique(GD.index)

        ## Preping figs
        fig0,ax0 = plt.subplots(dpi=250) 
        fig0.suptitle('Evolution of median distance to mean for ' + Expname[j])
        ax0.set_xlabel('Time after dormancy exit (hours)')
        ax0.set_ylabel('Median distance to mean contour (µm)')
        
        
        fig1,ax1 = plt.subplots(dpi=250) 
        fig1.suptitle('Evolution of median distance to normalized mean for ' + Expname[j])
        ax1.set_xlabel('Time after dormancy exit (hours)')
        ax1.set_ylabel('Median distance to mean contour (norm)')
        
        
        fig2,ax2 = plt.subplots(dpi=250) 
        fig2.suptitle('Evolution of L2 distance to mean for ' + Expname[j])
        ax2.set_xlabel('Time after dormancy exit (hours)')
        ax2.set_ylabel('L2 distance to mean contour (µm)')
        
        
        fig3,ax3 = plt.subplots(dpi=250) 
        fig3.suptitle('Evolution of L2 distance to normalized mean for ' + Expname[j])
        ax3.set_xlabel('Time after dormancy exit (hours)')
        ax3.set_ylabel('L2 distance to mean contour (norm)')
        
        
        fig4,ax4 = plt.subplots(dpi=250) 
        fig4.suptitle('Evolution of median distance to mean (not aligned in time) for ' + Expname[j])
        ax4.set_xlabel('Time after experiment start (hours)')
        ax4.set_ylabel('Median distance to mean contour (µm)')
        
        
        fig5,ax5 = plt.subplots(dpi=250) 
        fig5.suptitle('Evolution of median distance to normalized mean (not aligned in time) for ' + Expname[j])
        ax5.set_xlabel('Time after experiment start (hours)')
        ax5.set_ylabel('Median distance to mean contour (norm)')
        
        
        fig6,ax6 = plt.subplots(dpi=250) 
        fig6.suptitle('Evolution of L2 distance to mean (not aligned in time) for ' + Expname[j])
        ax6.set_xlabel('Time after experiment start (hours)')
        ax6.set_ylabel('L2 distance to mean contour (µm)')
        
        
        fig7,ax7 = plt.subplots(dpi=250) 
        fig7.suptitle('Evolution of L2 distance to normalized mean (not aligned in time) for ' + Expname[j])
        ax7.set_xlabel('Time after experiment start (hours)')
        ax7.set_ylabel('L2 distance to mean contour (norm)')
        
        
        plotprops = {'color':'white'}
        boxprops = {'color':'white','facecolor':colors[j]}

        grouping = []
        grouping_NotAl = []
        
        fullMedDist = []
        fullMedDistN = []
        fullL2Dist = []
        fullL2DistN = []
        
        fullMedDist_NotAl = []
        fullMedDistN_NotAl = []
        fullL2Dist_NotAl = []
        fullL2DistN_NotAl = []
        
        cnt = 0
        
        ## Main code
        for t in Times:
            
            MedDist = GD.loc[(GD['MeanCimg']==t*2),'MedDist'].values
            MedDistN = GD.loc[(GD['MeanCimg']==t*2),'MedDistNorm'].values
            L2Dist = GD.loc[(GD['MeanCimg']==t*2),'L2Dist'].values
            L2DistN = GD.loc[(GD['MeanCimg']==t*2),'L2DistNorm'].values
            
            fullMedDist = np.append(fullMedDist,MedDist)
            fullMedDistN = np.append(fullMedDistN,MedDistN)
            fullL2Dist = np.append(fullL2Dist,L2Dist)
            fullL2DistN = np.append(fullL2DistN,L2DistN)
            
            MedDist_NotAl = GD.loc[(GD['Img']==t*2),'NotalMedDist'].values
            MedDistN_NotAl = GD.loc[(GD['Img']==t*2),'NotalMedDistNorm'].values
            L2Dist_NotAl = GD.loc[(GD['Img']==t*2),'NotalL2Dist'].values
            L2DistN_NotAl = GD.loc[(GD['Img']==t*2),'NotalL2DistNorm'].values
            
            fullMedDist_NotAl = np.append(fullMedDist_NotAl,MedDist_NotAl)
            fullMedDistN_NotAl = np.append(fullMedDistN_NotAl,MedDistN_NotAl)
            fullL2Dist_NotAl = np.append(fullL2Dist_NotAl,L2Dist_NotAl)
            fullL2DistN_NotAl = np.append(fullL2DistN_NotAl,L2DistN_NotAl)
            
            grouping = np.append(grouping,np.ones(len(MedDist))*t)
            grouping_NotAl = np.append(grouping_NotAl,np.ones(len(MedDist_NotAl))*t)

            ax0.boxplot(MedDist, positions = [cnt], patch_artist = True, widths = [0.7],
                        boxprops=boxprops, capprops =plotprops,
                        showfliers = False,whiskerprops=plotprops,medianprops =plotprops)
            
            ax1.boxplot(MedDistN, positions = [cnt], patch_artist = True, widths = [0.7],
                        boxprops=boxprops, capprops =plotprops,
                        showfliers = False,whiskerprops=plotprops,medianprops =plotprops)  

            ax2.boxplot(L2Dist, positions = [cnt], patch_artist = True, widths = [0.7],
                        boxprops=boxprops, capprops =plotprops,
                        showfliers = False,whiskerprops=plotprops,medianprops =plotprops)
            
            ax3.boxplot(L2DistN, positions = [cnt], patch_artist = True, widths = [0.7],
                        boxprops=boxprops, capprops =plotprops,
                        showfliers = False,whiskerprops=plotprops,medianprops =plotprops)  

            
            ax4.boxplot(MedDist_NotAl, positions = [cnt], patch_artist = True, widths = [0.7],
                        boxprops=boxprops, capprops =plotprops,
                        showfliers = False,whiskerprops=plotprops,medianprops =plotprops)
            
            ax5.boxplot(MedDistN_NotAl, positions = [cnt], patch_artist = True, widths = [0.7],
                        boxprops=boxprops, capprops =plotprops,
                        showfliers = False,whiskerprops=plotprops,medianprops =plotprops)  

            ax6.boxplot(L2Dist_NotAl, positions = [cnt], patch_artist = True, widths = [0.7],
                        boxprops=boxprops, capprops =plotprops,
                        showfliers = False,whiskerprops=plotprops,medianprops =plotprops)
            
            ax7.boxplot(L2DistN_NotAl, positions = [cnt], patch_artist = True, widths = [0.7],
                        boxprops=boxprops, capprops =plotprops,
                        showfliers = False,whiskerprops=plotprops,medianprops =plotprops)            
            
            cnt = cnt+1
            
        sns.swarmplot(x=grouping,y=fullMedDist,color = 'lightgray', size=2, ax = ax0)
        ax0.set_xlim(left=-0.7,right=cnt-0.3)
        ax0.set_xticklabels(Times)
        
        sns.swarmplot(x=grouping,y=fullMedDistN,color = 'lightgray', size=2, ax = ax1)
        ax1.set_xlim(left=-0.7,right=cnt-0.3)
        ax1.set_xticklabels(Times)
            
        sns.swarmplot(x=grouping,y=fullL2Dist,color = 'lightgray', size=2, ax = ax2)
        ax2.set_xlim(left=-0.7,right=cnt-0.3)
        ax2.set_xticklabels(Times)
        
        sns.swarmplot(x=grouping,y=fullL2DistN,color = 'lightgray', size=2, ax = ax3)
        ax3.set_xlim(left=-0.7,right=cnt-0.3)
        ax3.set_xticklabels(Times)
            
        sns.swarmplot(x=grouping_NotAl,y=fullMedDist_NotAl,color = 'lightgray', size=2, ax = ax4)
        ax4.set_xlim(left=-0.7,right=cnt-0.3)
        ax4.set_xticklabels(Times)
        
        sns.swarmplot(x=grouping_NotAl,y=fullMedDistN_NotAl,color = 'lightgray', size=2, ax = ax5)
        ax5.set_xlim(left=-0.7,right=cnt-0.3)
        ax5.set_xticklabels(Times)
            
        sns.swarmplot(x=grouping_NotAl,y=fullL2Dist_NotAl,color = 'lightgray', size=2, ax = ax6)
        ax6.set_xlim(left=-0.7,right=cnt-0.3)
        ax6.set_xticklabels(Times)
        
        sns.swarmplot(x=grouping_NotAl,y=fullL2DistN_NotAl,color = 'lightgray', size=2, ax = ax7)
        ax7.set_xlim(left=-0.7,right=cnt-0.3)
        ax7.set_xticklabels(Times)
        
        fig0.savefig(fullP + 'MedDistToMean.png')
        fig1.savefig(fullP + 'MedDistToMeanNorm.png')
        fig2.savefig(fullP + 'L2DistToMean.png')
        fig3.savefig(fullP + 'L2DistToMeanNorm.png')
        
        fig4.savefig(fullP + 'MedDistToMeanNotAl.png')
        fig5.savefig(fullP + 'MedDistToMeanNormNotAl.png')
        fig6.savefig(fullP + 'L2DistToMeanNotAl.png')
        fig7.savefig(fullP + 'L2DistToMeanNormNotAl.png')
    
    
        
        
     ### Comparison between experiments    
#     if len(GDs)>1:       
#         for t in Times:
#             ## Preping figs
#             fig1,ax1 = plt.subplots(dpi=250) 
#             fig1.suptitle('Median distance to mean ' + str(t) + ' hours after dormancy exit')
            
#             fig2,ax2 = plt.subplots(dpi=250) 
#             fig2.suptitle('Median distance to normalized mean ' + str(t) + ' hours after dormancy exit')

#             MedDist = [None]*len(GDs)
#             MedDistN = [None]*len(GDs)
#             cap = [None]*len(GDs)
#             capN = [None]*len(GDs)
#             grouping = []
#             fullMedDist = []
#             fullMedDistN = []
#             cnt = 0

#             for j in range(len(Expname)):
#                 GD = GDs[j]
#                 plotprops = {'color':'white'}
#                 boxprops = {'color':'white','facecolor':colors[j]}

#                 MedDist[j] = GD.loc[(GD['MeanCimg']==t*2),'MedDist']
#                 MedDistN[j] = GD.loc[(GD['MeanCimg']==t*2),'MedDistNorm']

#                 fullMedDist = np.append(fullMedDist,MedDist[j].values)
#                 fullMedDistN = np.append(fullMedDistN,MedDistN[j].values)
#                 grouping = np.append(grouping,np.ones(len(MedDist[j]))*j)

#                 bp = ax1.boxplot(MedDist[j], positions = [cnt], patch_artist = True, widths = [0.7],
#                             boxprops=boxprops, capprops =plotprops,
#                             showfliers = False,whiskerprops=plotprops,medianprops =plotprops)

#                 bpN = ax2.boxplot(MedDistN[j], positions = [cnt], patch_artist = True, widths = [0.7],
#                             boxprops=boxprops, capprops =plotprops,
#                             showfliers = False,whiskerprops=plotprops,medianprops =plotprops)
#                 cnt = cnt+1

#                 cap[j] = bp['caps'][1].get_yMedDist(orig=True)[0]
#                 capN[j] = bpN['caps'][1].get_yMedDist(orig=True)[0]

#             sns.swarmplot(x=grouping,y=fullMedDist,color = 'lightgray', size=2, ax = ax1)
#             ax1.set_xlim(left=-0.7,right=cnt-0.3)
#             ax1.set_xticklabels(Expname)

#             sns.swarmplot(x=grouping,y=fullMedDistN,color = 'lightgray', size=2, ax = ax2)
#             ax2.set_xlim(left=-0.7,right=cnt-0.3)
#             ax2.set_xticklabels(Expname)

#             step = np.max(cap)*0.125
#             stepN = np.max(capN)*0.125
#             fullstep = 0
#             fullstepN = 0
#             hmax = np.max(cap)
#             hmaxN = np.max(capN)


#             for i in range(len(GDs)-1):
#                 for j in range(i+1,len(GDs)):

#                     fullstep = plotSig(ax1,hmax,step,fullstep,MedDist[i],MedDist[j],i,j)
#                     fullstepN = plotSig(ax2,hmaxN,stepN,fullstepN,MedDistN[i],MedDistN[j],i,j)

#             fig1.savefig(Pfolder + '_'.join(Expname) + '_t=' + str(t) + 'h_MedDistToMean.png')
#             fig2.savefig(Pfolder + '_'.join(Expname) + '_t=' + str(t) + 'h_MedDistToMeanNorm.png')

### Plot distances between mean contours from different experiments

In [ ]:
def DistBetwMean(Pfig,CDMeans,Expnames):
    N = len(CDMeans)
    
    Pfolder = Pfig + '\\MeanContoursDistance\\'
    Pfolder2 = Pfolder + '\\2by2\\'
    
    AllLabels = []
    
    if not os.path.exists(Pfolder):
        os.mkdir(Pfolder) # create general folder  
    if not os.path.exists(Pfolder2):
        os.mkdir(Pfolder2) # create general folder  
    
    fig00,ax00 = plt.subplots(dpi=350)
    fig00.suptitle('Median distance between mean contours')
    ax00.set_xlabel('Time after dormancy exit (hours)')
    ax00.set_ylabel('Distance (µm)')

    fig01,ax01 = plt.subplots(dpi=350)
    fig01.suptitle('Median distance between normalized mean contours')
    ax01.set_xlabel('Time after dormancy exit (hours)')
    ax01.set_ylabel('Distance (norm)')

    fig02,ax02 = plt.subplots(dpi=350)
    fig02.suptitle('Median distance between mean contours (not time aligned)')
    ax02.set_xlabel('Time after dormancy exit (hours)')
    ax02.set_ylabel('Distance (µm)')

    fig03,ax03 = plt.subplots(dpi=350)
    fig03.suptitle('Median distance between normalized mean contours (not time aligned)')
    ax03.set_xlabel('Time after dormancy exit (hours)')
    ax03.set_ylabel('Distance (norm)')

    fig04,ax04 = plt.subplots(dpi=350)
    fig04.suptitle('L2 distance between mean contours')
    ax04.set_xlabel('Time after experiment start (hours)')
    ax04.set_ylabel('Distance (µm)')

    fig05,ax05 = plt.subplots(dpi=350)
    fig05.suptitle('L2 distance between normalized mean contours')
    ax05.set_xlabel('Time after experiment start (hours)')
    ax05.set_ylabel('Distance (norm)')

    fig06,ax06 = plt.subplots(dpi=350)
    fig06.suptitle('L2 distance between mean contours (not time aligned)')
    ax06.set_xlabel('Time after experiment start (hours)')
    ax06.set_ylabel('Distance (µm)')

    fig07,ax07 = plt.subplots(dpi=350)
    fig07.suptitle('L2 distance between normalized mean contours (not time aligned)')
    ax07.set_xlabel('Time after experiment start (hours)')
    ax07.set_ylabel('Distance (norm)')
    
    for i in range(0,N-1):
        for j in range(i+1,N):
            
            label = Expnames[i] + ' & ' + Expnames[j]
            AllLabels = np.append(AllLabels,label)
            
            fig0,ax0 = plt.subplots(dpi=350)
            fig0.suptitle('Median distance between mean contours from \n' + label)
            ax0.set_xlabel('Time after dormancy exit (hours)')
            ax0.set_ylabel('Distance (µm)')
            
            fig1,ax1 = plt.subplots(dpi=350)
            fig1.suptitle('Median distance between normalized mean contours from \n' + label)
            ax1.set_xlabel('Time after dormancy exit (hours)')
            ax1.set_ylabel('Distance (norm)')
            
            fig2,ax2 = plt.subplots(dpi=350)
            fig2.suptitle('Median distance between mean contours (not time aligned) from \n' + label)
            ax2.set_xlabel('Time after dormancy exit (hours)')
            ax2.set_ylabel('Distance (µm)')
            
            fig3,ax3 = plt.subplots(dpi=350)
            fig3.suptitle('Median distance between normalized mean contours (not time aligned) from \n' + label)
            ax3.set_xlabel('Time after dormancy exit (hours)')
            ax3.set_ylabel('Distance (norm)')
            
            fig4,ax4 = plt.subplots(dpi=350)
            fig4.suptitle('L2 distance between mean contours from \n' + label)
            ax4.set_xlabel('Time after experiment start (hours)')
            ax4.set_ylabel('Distance (µm)')
            
            fig5,ax5 = plt.subplots(dpi=350)
            fig5.suptitle('L2 distance between normalized mean contours from \n' + label)
            ax5.set_xlabel('Time after experiment start (hours)')
            ax5.set_ylabel('Distance (norm)')
            
            fig6,ax6 = plt.subplots(dpi=350)
            fig6.suptitle('L2 distance between mean contours (not time aligned) from \n' + label)
            ax6.set_xlabel('Time after experiment start (hours)')
            ax6.set_ylabel('Distance (µm)')
            
            fig7,ax7 = plt.subplots(dpi=350)
            fig7.suptitle('L2 distance between normalized mean contours (not time aligned) from \n' + label)
            ax7.set_xlabel('Time after experiment start (hours)')
            ax7.set_ylabel('Distance (norm)')
            
            CD1 = CDMeans[i]
            CD2 = CDMeans[j]
            
############################################ Contours aligned in time ##############################################
            nimg = int(np.min([np.max(CD1.loc['meanC','Img']),np.max(CD2.loc['meanC','Img'])]))+1
            
            Dist = np.empty(nimg)
            DistN = np.empty(nimg)            
            L2 = np.empty(nimg)
            L2N = np.empty(nimg)
            Time = np.empty(nimg)
            
            for k in range(nimg):
        
                X1 = CD1.loc[(CD1.index == 'meanC') & (CD1['Img']==k),'X'].values
                Y1 = CD1.loc[(CD1.index == 'meanC') & (CD1['Img']==k),'Y'].values
        
                X2 = CD2.loc[(CD2.index == 'meanC') & (CD2['Img']==k),'X'].values
                Y2 = CD2.loc[(CD2.index == 'meanC') & (CD2['Img']==k),'Y'].values
                
                
                AllDists = vf.dist(X1,Y1,X2,Y2)
                Dist[k] = np.median(AllDists)
                L2[k] = np.sqrt(np.sum(np.square(AllDists)))/len(AllDists)
                Time[k] = k/2
                
                X1N,Y1N = vf.normaliseContourByArea(X1,Y1)
                X2N,Y2N = vf.normaliseContourByArea(X2,Y2)
                                
                AllDistsN = vf.dist(X1N,Y1N,X2N,Y2N)
                DistN[k] = np.median(AllDistsN)
                L2N[k] = np.sqrt(np.sum(np.square(AllDistsN)))/len(AllDistsN)
            
            ax0.plot(Time,Dist,'-oc',ms=2,lw=1)
            ax00.plot(Time,Dist,'-o',ms=2,lw=1, label = label)
            fig0.savefig(Pfolder2 + 'MedDistBetween' + Expnames[i] + '_&_' + Expnames[j] + '.png')
            
            ax1.plot(Time,DistN,'-oc',ms=2,lw=1)
            ax01.plot(Time,DistN,'-o',ms=2,lw=1, label = label)
            fig1.savefig(Pfolder2 + 'MedDistBetween' + Expnames[i] + '_&_' + Expnames[j] + '_Norm.png')
            
            ax4.plot(Time,L2,'-oc',ms=2,lw=1)
            ax04.plot(Time,L2,'-o',ms=2,lw=1, label = label)
            fig4.savefig(Pfolder2 + 'L2DistBetween' + Expnames[i] + '_&_' + Expnames[j] + '.png')
            
            ax5.plot(Time,L2N,'-oc',ms=2,lw=1)
            ax05.plot(Time,L2N,'-o',ms=2,lw=1, label = label)
            fig5.savefig(Pfolder2 + 'L2DistBetween' + Expnames[i] + '_&_' + Expnames[j] + '_Norm.png')
            
            
            
############################################ Contours not aligned in time ##############################################
            nimg = int(np.min([np.max(CD1.loc['meanCnotal','Img']),np.max(CD2.loc['meanCnotal','Img'])]))+1
            
            Dist = np.empty(nimg)
            DistN = np.empty(nimg)            
            L2 = np.empty(nimg)
            L2N = np.empty(nimg)
            Time = np.empty(nimg)
            
            for k in range(nimg):
        
                X1 = CD1.loc[(CD1.index == 'meanCnotal') & (CD1['Img']==k),'X'].values
                Y1 = CD1.loc[(CD1.index == 'meanCnotal') & (CD1['Img']==k),'Y'].values
        
                X2 = CD2.loc[(CD2.index == 'meanCnotal') & (CD2['Img']==k),'X'].values
                Y2 = CD2.loc[(CD2.index == 'meanCnotal') & (CD2['Img']==k),'Y'].values
                
                
                AllDists = vf.dist(X1,Y1,X2,Y2)
                Dist[k] = np.median(AllDists)
                L2[k] = np.sqrt(np.sum(np.square(AllDists)))/len(AllDists)
                Time[k] = k/2
                
                X1N,Y1N = vf.normaliseContourByArea(X1,Y1)
                X2N,Y2N = vf.normaliseContourByArea(X2,Y2)
                                
                AllDistsN = vf.dist(X1N,Y1N,X2N,Y2N)
                DistN[k] = np.median(AllDistsN)
                L2N[k] = np.sqrt(np.sum(np.square(AllDistsN)))/len(AllDistsN)
            
            ax2.plot(Time,Dist,'-oc',ms=2,lw=1)
            ax02.plot(Time,Dist,'-o',ms=2,lw=1, label = label)
            fig2.savefig(Pfolder2 + 'MedDistBetween' + Expnames[i] + '_&_' + Expnames[j] + '_NotAl.png')
            
            ax3.plot(Time,DistN,'-oc',ms=2,lw=1)
            ax03.plot(Time,DistN,'-o',ms=2,lw=1, label = label)
            fig3.savefig(Pfolder2 + 'MedDistBetween' + Expnames[i] + '_&_' + Expnames[j] + '_NotAl_Norm.png')
            
            ax6.plot(Time,L2,'-oc',ms=2,lw=1)
            ax06.plot(Time,L2,'-o',ms=2,lw=1, label = label)
            fig6.savefig(Pfolder2 + 'L2DistBetween' + Expnames[i] + '_&_' + Expnames[j] + '_NotAl.png')
            
            ax7.plot(Time,L2N,'-oc',ms=2,lw=1)
            ax07.plot(Time,L2N,'-o',ms=2,lw=1, label = label)
            fig7.savefig(Pfolder2 + 'L2DistBetween' + Expnames[i] + '_&_' + Expnames[j] + '_NotAl_Norm.png')
          
    ax00.legend()
    fig00.savefig(Pfolder + 'MedDist.png')
    ax01.legend()   
    fig01.savefig(Pfolder + 'MedDist_Norm.png')
    ax04.legend()        
    fig04.savefig(Pfolder + 'L2Dist.png')
    ax05.legend()      
    fig05.savefig(Pfolder + 'L2Dist_Norm.png')
    ax02.legend()       
    fig02.savefig(Pfolder + 'MedDist_NotAl.png')
    ax03.legend()       
    fig03.savefig(Pfolder + 'MedDist_NotAl_Norm.png')
    ax06.legend()       
    fig06.savefig(Pfolder + 'L2Dist_NotAl.png')
    ax07.legend()       
    fig07.savefig(Pfolder + 'L2Dist_NotAl_Norm.png')

### Compute and plot ranksum stats

In [ ]:
def plotSig(ax,hmax,step,fullstep,data1,data2,pos1,pos2):
    
    s,p = ranksums(data1,data2)
                        
#     if pos2 == pos1+1: 
#         h = np.median([np.median(data1),np.median(data2)])
#         ax.plot([pos1+0.2, pos2-0.2], [h ,h], 'w-',zorder=0)
#         ax.text((pos1+pos2)/2,h+0.2*step,'p = ' + str(round(p*1000)/1000), ha='center',fontsize='small')
#         ax.set_ylim(top=hmax+fullstep+step)
            
#     else:       
    h = hmax
    fullstep += step
    ax.plot([pos1, pos2], [h+fullstep ,h+fullstep], 'w-',zorder=0)
    ax.text((pos1+pos2)/2,h+fullstep+0.2*step,'p = ' + str(round(p*1000)/1000), ha='center',fontsize='small')
    ax.set_ylim(top=h+fullstep+step)

    return(fullstep)

## Enter data and run

### Data details

In [ ]:
%run D:/Users/laplaud/Desktop/PostDoc/Code/JupyterNB/ExperimentList.py

### Loading data 

In [ ]:

## Experience Degas V6 Stade1 du 24/01/2022 (Growth only, Ct1 & Ct2)

ContourData220124_Ct1 = pd.read_csv(P220124_Ct1 + '\\ContourData220124_Ct1_ParamAligned.csv', index_col = 'Ind')
GlobalData220124_Ct1 = pd.read_csv(P220124_Ct1 + '\\GlobalData220124_Ct1_ParamAligned.csv', index_col = 'Ind') 
Fitdata220124_Ct1 = pd.read_csv(P220124_Ct1 + '\\GlobalData220124_Ct1_AreaContFit.csv', index_col = 'Ind')
Tstarts220124_Ct1 = Fitdata220124_Ct1.loc[Fitdata220124_Ct1['Img']==0,'tdebShift']
StackList220124_Ct1 = np.unique(GlobalData220124_Ct1.index)


ContourData220124_Ct2 = pd.read_csv(P220124_Ct2 + '\\ContourData220124_Ct2_ParamAligned.csv', index_col = 'Ind')
GlobalData220124_Ct2 = pd.read_csv(P220124_Ct2 + '\\GlobalData220124_Ct2_ParamAligned.csv', index_col = 'Ind')  
Fitdata220124_Ct2 = pd.read_csv(P220124_Ct2 + '\\GlobalData220124_Ct2_AreaContFit.csv', index_col = 'Ind')
Tstarts220124_Ct2 = Fitdata220124_Ct2.loc[Fitdata220124_Ct2['Img']==0,'tdebShift']
StackList220124_Ct2 = np.unique(Fitdata220124_Ct2.index)


# ## Experience Degas V6 Stade1 du 14/02/2022 (Growth + OC, Ct1 & Ct2)

# # Growth
ContourData220214_Ct1 = pd.read_csv(P220214_Ct1 + '\\ContourData220214_Ct1_ParamAligned.csv', index_col = 'Ind')
GlobalData220214_Ct1 = pd.read_csv(P220214_Ct1 + '\\GlobalData220214_Ct1_ParamAligned.csv', index_col = 'Ind') 
Fitdata220214_Ct1 = pd.read_csv(P220214_Ct1 + '\\GlobalData220214_Ct1_AreaContFit.csv', index_col = 'Ind')
Tstarts220214_Ct1 = Fitdata220214_Ct1.loc[Fitdata220214_Ct1['Img']==0,'tdebShift']
StackList220214_Ct2 = np.unique(Fitdata220214_Ct1.index)

ContourData220214_Ct2 = pd.read_csv(P220214_Ct2 + '\\ContourData220214_Ct2_ParamAligned.csv', index_col = 'Ind')
GlobalData220214_Ct2 = pd.read_csv(P220214_Ct2 + '\\GlobalData220214_Ct2_ParamAligned.csv', index_col = 'Ind') 
Fitdata220214_Ct2 = pd.read_csv(P220214_Ct2 + '\\GlobalData220214_Ct2_AreaContFit.csv', index_col = 'Ind')
Tstarts220214_Ct2 = Fitdata220214_Ct2.loc[Fitdata220214_Ct2['Img']==0,'tdebShift']
StackList220214_Ct2 = np.unique(Fitdata220214_Ct2.index)

# # Osmotic Chocs
# ContourData220214_Ct1_Osmo = pd.read_csv(P220214_Ct1_Osmo + '\\ContourData220214_Ct1_Osmo_ParamAligned.csv', index_col = 'Ind')
# GlobalData220214_Ct1_Osmo = pd.read_csv(P220214_Ct1_Osmo + '\\GlobalData220214_Ct1_Osmo_ParamAligned.csv', index_col = 'Ind') 

# ContourData220214_Ct2_Osmo = pd.read_csv(P220214_Ct2 + '\\ContourData220214_Ct2_Osmo_ParamAligned.csv', index_col = 'Ind')
# GlobalData220214_Ct2_Osmo = pd.read_csv(P220214_Ct2_Osmo + '\\GlobalData220214_Ct2_Osmo_ParamAligned.csv', index_col = 'Ind')  



print('Parametrised contours loaded. \n')


### Compute contour characteristics

#### Mean contours

In [ ]:
# # ## Experience Degas V6 Stade1 du 24/01/2022 (Growth only, Ct1 & Ct2)
# MeanCD220124_Ct1,MeanGD220124_Ct1 =  computeMeanContourTime(ContourData220124_Ct1,
#                                                             GlobalData220124_Ct1,Tstarts220124_Ct1)
# MeanGD220124_Ct1.to_csv(P220124_Ct1 + '\\GlobalData220124_Ct1_MeanC.csv',index_label = 'Ind')
# MeanCD220124_Ct1.to_csv(P220124_Ct1 + '\\ContourData220124_Ct1_MeanC.csv',index_label = 'Ind')
# GlobalData220124_Ct1.to_csv(P220124_Ct1 + '\\GlobalData220124_Ct1_ParamAligned.csv',index_label = 'Ind')
# ContourData220124_Ct1.to_csv(P220124_Ct1 + '\\ContourData220124_Ct1_ParamAligned.csv',index_label = 'Ind')

# MeanCD220124_Ct2,MeanGD220124_Ct2 =  computeMeanContourTime(ContourData220124_Ct2, 
#                                                             GlobalData220124_Ct2,Tstarts220124_Ct2)
# MeanGD220124_Ct2.to_csv(P220124_Ct2 + '\\GlobalData220124_Ct2_MeanC.csv',index_label = 'Ind')
# MeanCD220124_Ct2.to_csv(P220124_Ct2 + '\\ContourData220124_Ct2_MeanC.csv',index_label = 'Ind')
# GlobalData220124_Ct2.to_csv(P220124_Ct2 + '\\GlobalData220124_Ct2_ParamAligned.csv',index_label = 'Ind')
# ContourData220124_Ct2.to_csv(P220124_Ct2 + '\\ContourData220124_Ct2_ParamAligned.csv',index_label = 'Ind')


# # ## Experience Degas V6 Stade1 du 14/02/2022 (Growth + OC, Ct1 & Ct2)

# MeanCD220214_Ct1,MeanGD220214_Ct1 =  computeMeanContourTime(ContourData220214_Ct1,
#                                                             GlobalData220214_Ct1,Tstarts220214_Ct1)
# MeanGD220214_Ct1.to_csv(P220214_Ct1 + '\\GlobalData220214_Ct1_MeanC.csv',index_label = 'Ind')
# MeanCD220214_Ct1.to_csv(P220214_Ct1 + '\\ContourData220214_Ct1_MeanC.csv',index_label = 'Ind')
# GlobalData220214_Ct1.to_csv(P220214_Ct1 + '\\GlobalData220214_Ct1_ParamAligned.csv',index_label = 'Ind')
# ContourData220214_Ct1.to_csv(P220214_Ct1 + '\\ContourData220214_Ct1_ParamAligned.csv',index_label = 'Ind')

# MeanCD220214_Ct2,MeanGD220214_Ct2 =  computeMeanContourTime(ContourData220214_Ct2, 
#                                                             GlobalData220214_Ct2,Tstarts220214_Ct2)
# MeanGD220214_Ct2.to_csv(P220214_Ct2 + '\\GlobalData220214_Ct2_MeanC.csv',index_label = 'Ind')
# MeanCD220214_Ct2.to_csv(P220214_Ct2 + '\\ContourData220214_Ct2_MeanC.csv',index_label = 'Ind')
# GlobalData220214_Ct2.to_csv(P220214_Ct2 + '\\GlobalData220214_Ct2_ParamAligned.csv',index_label = 'Ind')
# ContourData220214_Ct2.to_csv(P220214_Ct2 + '\\ContourData220214_Ct2_ParamAligned.csv',index_label = 'Ind')


#### Distance to mean contour + plots

In [ ]:
# Load mean contours contour data

MeanCD220124_Ct1 = pd.read_csv(P220124_Ct1 + '\\ContourData220124_Ct1_MeanC.csv', index_col = 'Ind')
MeanCD220124_Ct2 = pd.read_csv(P220124_Ct2 + '\\ContourData220124_Ct2_MeanC.csv', index_col = 'Ind')

MeanCD220214_Ct1 = pd.read_csv(P220214_Ct1 + '\\ContourData220214_Ct1_MeanC.csv', index_col = 'Ind')
MeanCD220214_Ct2 = pd.read_csv(P220214_Ct2 + '\\ContourData220214_Ct2_MeanC.csv', index_col = 'Ind')

In [ ]:
## Experience Degas V6 Stade1 du 24/01/2022 (Growth only, Ct1 & Ct2)

DistToMean(PFig,ContourData220124_Ct1,GlobalData220124_Ct1,MeanCD220124_Ct1,'220124_Ct1',
          doPlots = False, showPlots = False)
GlobalData220124_Ct1.to_csv(P220124_Ct1 + '\\GlobalData220124_Ct1_ParamAligned.csv',index_label = 'Ind')

DistToMean(PFig,ContourData220124_Ct2,GlobalData220124_Ct2,MeanCD220124_Ct2,'220124_Ct2',
          doPlots = False, showPlots = False)
GlobalData220124_Ct2.to_csv(P220124_Ct2 + '\\GlobalData220124_Ct2_ParamAligned.csv',index_label = 'Ind')

## Experience Degas V6 Stade1 du 14/02/2022 (Growth + OC, Ct1 & Ct2)
DistToMean(PFig,ContourData220214_Ct1,GlobalData220214_Ct1,MeanCD220214_Ct1,'220214_Ct1',
          doPlots = False, showPlots = False)
GlobalData220214_Ct1.to_csv(P220214_Ct1 + '\\GlobalData220214_Ct1_ParamAligned.csv',index_label = 'Ind')

DistToMean(PFig,ContourData220214_Ct2,GlobalData220214_Ct2,MeanCD220214_Ct2,'220214_Ct2',
          doPlots = False, showPlots = False)
GlobalData220214_Ct2.to_csv(P220214_Ct2 + '\\GlobalData220214_Ct2_ParamAligned.csv',index_label = 'Ind')

### Plot results

#### Colors definition

In [ ]:
# Colors

LightGreen = [0.5, 1, 0.3]
Green = [0.3, 0.7, 0.2]
DarkGreen = [0.1, 0.4, 0.1]
PastelGreen = [0.5, 1, 0.8]
DarkPastelGreen = [0.2, 0.6, 0.5]

Pink = [1, 0.3, 0.5]
Red = [0.8, 0.2, 0.2]
DarkRed = [0.6, 0.1, 0.2]

LightBlue = [0.3, 0.5, 1]
Blue = [0.2, 0.3, 0.7]
DarkBlue = [0.1, 0.1, 0.4]

DarkPurple = [0.4, 0, 0.3]
Purple = [0.55, 0, 0.45]
LightPurple = [0.8, 0, 0.7]

Yellow = [0.8, 0.8, 0.2]
DarkYellow = [0.5, 0.5, 0.2]


#### Load quantifications

#### Plots individual contours in time

In [ ]:
# ## Experience Degas V6 Stade1 du 24/01/2022 (Growth only, Ct1 & Ct2)
# plotIndiContours(PFig,ContourData220124_Ct1, '220124_Ct1', showplots=True)

# plotIndiContours(PFig,MeanCD220124_Ct1, '220124_Ct1', showplots=True)

# plotIndiContours(PFig,ContourData220124_Ct2, '220124_Ct2', showplots=True)

# plotIndiContours(PFig,MeanCD220124_Ct2, '220124_Ct2', showplots=True)

## Experience Degas V6 Stade1 du 14/02/2022 (Growth only, Ct1 & Ct2)
# plotIndiContours(PFig, ContourData220214_Ct1, '220214_Ct1', showplots=True)

# plotIndiContours(PFig,MeanCD220214_Ct1, '220214_Ct1', showplots=True)

# plotIndiContours(PFig, ContourData220214_Ct2, '220214_Ct2', showplots=True)

# plotIndiContours(PFig,MeanCD220214_Ct2, '220214_Ct2', showplots=True)

#### Plot distances relative to mean contour

##### Load mean contours data

In [ ]:
# Load mean & contours contour data

MeanCD220124_Ct1 = pd.read_csv(P220124_Ct1 + '\\ContourData220124_Ct1_MeanC.csv', index_col = 'Ind')
GlobalData220124_Ct1 = pd.read_csv(P220124_Ct1 + '\\GlobalData220124_Ct1_ParamAligned.csv', index_col = 'Ind') 

MeanCD220124_Ct2 = pd.read_csv(P220124_Ct2 + '\\ContourData220124_Ct2_MeanC.csv', index_col = 'Ind')
GlobalData220124_Ct2 = pd.read_csv(P220124_Ct2 + '\\GlobalData220124_Ct2_ParamAligned.csv', index_col = 'Ind') 

MeanCD220214_Ct1 = pd.read_csv(P220214_Ct1 + '\\ContourData220214_Ct1_MeanC.csv', index_col = 'Ind')
GlobalData220214_Ct1 = pd.read_csv(P220214_Ct1 + '\\GlobalData220214_Ct1_ParamAligned.csv', index_col = 'Ind') 

MeanCD220214_Ct2 = pd.read_csv(P220214_Ct2 + '\\ContourData220214_Ct2_MeanC.csv', index_col = 'Ind')
GlobalData220214_Ct2 = pd.read_csv(P220214_Ct2 + '\\GlobalData220214_Ct2_ParamAligned.csv', index_col = 'Ind') 

##### Plot distance of individual contours to mean

In [ ]:
## Plots
plotMeanDist(PFig,[GlobalData220124_Ct1,GlobalData220124_Ct2],
             ['220124_Ct1','220124_Ct2'],[LightPurple, DarkPurple],range(0,25,4))

plotMeanDist(PFig,[GlobalData220214_Ct1,GlobalData220214_Ct2],
             ['220214_Ct1','220214_Ct2'],[LightBlue, DarkBlue],range(0,22,4))

plotMeanDist(PFig,[GlobalData220124_Ct1,GlobalData220124_Ct2,GlobalData220214_Ct1,GlobalData220214_Ct2],
             ['220124_Ct1','220124_Ct2','220214_Ct1','220214_Ct2'],[LightPurple, DarkPurple,LightBlue, DarkBlue],range(0,22,4))

##### Plot distance between mean contours

In [ ]:
DistBetwMean(PFig,[MeanCD220124_Ct1,MeanCD220124_Ct2,MeanCD220214_Ct1,MeanCD220214_Ct2,],
             ['220124_Ct1','220124_Ct2','220214_Ct1','220214_Ct2'])

## Test Zone